<a href="https://colab.research.google.com/github/amermahyoub/AutoML/blob/main/AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# !pip install streamlit -q

import streamlit as st
import base64
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_diabetes

In [25]:
import ipykernel
ipykernel.__version__

'5.1.2'

In [26]:
# !pip install streamlit -q

# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip -qq ngrok-stable-linux-amd64.zip

# get_ipython().system_raw('./ngrok http 8501 &')
# ! curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# !streamlit hello

In [27]:
# !pip uninstall ipykernel
# !pip install ipykernel==5.1.2
# !pip install pydeck


In [28]:
# Page layout

st.set_page_config(page_title='The Machine Learning Hyperparameter Optimization App',
                   layout='wide', page_icon='random')

st.write("""
# The Machine Learning Hyperparameter Optimization App
**(Regression Edition)**
In this implementation, the *RandomForestRegressor()* function is used in this app for build a regression model using the **Random Forest** algorithm.
""")

In [29]:
# Sidebar - Collects user input features into dataframe

st.sidebar.header('Upload your CSV data')
uploaded_file = st.sidebar.file_uploader('Upload your input CSV file', type='csv')
st.sidebar.markdown("""
[Example CSV input file](https://raw.githubusercontent.com/dataprofessor/data/master/delaney_solubility_with_descriptors.csv)
""")

DeltaGenerator(_root_container=1, _provided_cursor=None, _parent=DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None), _block_type=None, _form_data=None)

In [30]:
# Sidebar - Specify parameter settings

st.sidebar.header('Set Parameters')
spit_size = st.sidebar.slider('Data split ratior (% for Training set)', 
                              min_value=10, max_value=90, value=80, step=5)


In [31]:
st.subheader('Learning Parameters')
parameter_n_estimator = st.sidebar.slider('Number of estimators (n_estimators)', 0, 500)
parameter_n_estimator_step = st.sidebar.number_input('Step size for n_estomators', 10)
st.sidebar.write('---')

In [32]:
parameter_max_features = st.sidebar.slider('Max features (max_features)', 1, 50, (1,3), 1)
st.sidebar.number_input('Step size for max_features', 1)
st.sidebar.write('---')

In [33]:
parameter_min_samples_split = st.sidebar.slider('Minimum number of samples required to split an internal node (min_samples_split)'
                                                , 1, 10, 2, 1)
parameter_min_samples_leaf = st.sidebar.slider('Minimum number of samples required to be at a leaf node (min_samples_leaf)'
                                                , 1, 10, 2, 1)

In [34]:
st.sidebar.subheader('General Parameters')
parameter_random_state = st.sidebar.slider('Seed number (random_state)', 0, 1000, 42, 1)
parameter_criterion = st.sidebar.select_slider('Performance measure (criterion)', options=['mse', 'mae'])
parameter_bootstrap = st.sidebar.select_slider('Bootstrap samples when building trees (bootstrap)', options=[True, False])
parameter_oob_score = st.sidebar.select_slider('Whether to use out-of-bag samples to estimate the R^2 on unseen data (oob_score)', options=[False, True])
parameter_n_jobs = st.sidebar.select_slider('Number of jobs to run in parallel (n_jobs)', options=[1, -1])


In [41]:
n_estimators_range = np.arange(parameter_n_estimator[0], parameter_n_estimator[1]+parameter_n_estimator_step, parameter_n_estimator_step)
max_features_range = np.arange(parameter_max_features[0], parameter_max_features[1]+1, 1)
param_grid = dict(max_features=max_features_range, n_estimators=n_estimators_range)

0


In [ ]:
# Displays the dataset
st.subheader('Dataset')

In [ ]:
# Model building

def file_download(df):
  csv = df.to_csv(index=False)
  b64 = base64.b64encode(csv,encode()).decode() # strings <-> bytes conversions
  href = f'<a href="data:file/csv;base64,{b64}" download="model_performance.csv">Download CSV File</a>'
  return href

def build_model(df):
  x = df.loc[:,:-1] # Using all column except for the last column as X
  y = df.loc[:,-1] # Selecting the last column as Y

  st.markdown('A model is being built to predict the following **Y** variable:')
  st.info(y.name)

  # Data splitting
  X_train, X_test, Y_train, Y_test = train_test_split(y, x, test_size=split_size)

  # Defining our Random Forest model
  rf = RandomForestRegressor(n_estimators=parameter_n_estimators,
        random_state=parameter_random_state,
        max_features=parameter_max_features,
        criterion=parameter_criterion,
        min_samples_split=parameter_min_samples_split,
        min_samples_leaf=parameter_min_samples_leaf,
        bootstrap=parameter_bootstrap,
        oob_score=parameter_oob_score,
        n_jobs=parameter_n_jobs)
  
  grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
  grid_search.fit(X_train, Y_train)

  Y_pred_test = grid.predict(X_test)
    st.write('Coefficient of determination ($R^2$):')
    st.info( r2_score(Y_test, Y_pred_test) )

    st.write('Error (MSE or MAE):')
    st.info( mean_squared_error(Y_test, Y_pred_test) )

    st.write("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

    st.subheader('Model Parameters')
    st.write(grid.get_params())

    #-----Process grid data-----#
    grid_results = pd.concat([pd.DataFrame(grid.cv_results_["params"]),pd.DataFrame(grid.cv_results_["mean_test_score"], columns=["R2"])],axis=1)
    # Segment data into groups based on the 2 hyperparameters
    grid_contour = grid_results.groupby(['max_features','n_estimators']).mean()
    # Pivoting the data
    grid_reset = grid_contour.reset_index()
    grid_reset.columns = ['max_features', 'n_estimators', 'R2']
    grid_pivot = grid_reset.pivot('max_features', 'n_estimators')
    x = grid_pivot.columns.levels[1].values
    y = grid_pivot.index.values
    z = grid_pivot.values

    #-----Plot-----#
    layout = go.Layout(
            xaxis=go.layout.XAxis(
              title=go.layout.xaxis.Title(
              text='n_estimators')
             ),
             yaxis=go.layout.YAxis(
              title=go.layout.yaxis.Title(
              text='max_features')
            ) )
    fig = go.Figure(data= [go.Surface(z=z, y=y, x=x)], layout=layout )
    fig.update_layout(title='Hyperparameter tuning',
                      scene = dict(
                        xaxis_title='n_estimators',
                        yaxis_title='max_features',
                        zaxis_title='R2'),
                      autosize=False,
                      width=800, height=800,
                      margin=dict(l=65, r=50, b=65, t=90))
    st.plotly_chart(fig)

    #-----Save grid data-----#
    x = pd.DataFrame(x)
    y = pd.DataFrame(y)
    z = pd.DataFrame(z)
    df = pd.concat([x,y,z], axis=1)
    st.markdown(filedownload(grid_results), unsafe_allow_html=True)


In [ ]:
if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)
    st.write(df)
    build_model(df)
else:
    st.info('Awaiting for CSV file to be uploaded.')
    if st.button('Press to use Example Dataset'):
        diabetes = load_diabetes()
        X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
        Y = pd.Series(diabetes.target, name='response')
        df = pd.concat( [X,Y], axis=1 )

        st.markdown('The **Diabetes** dataset is used as the example.')
        st.write(df.head(5))

        build_model(df)